<a href="https://colab.research.google.com/github/monacofj/moeabench/blob/add-test/misc/benchmark_gallery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoeaBench: Scientific Audit Confrontation
This notebook provides a systematic visual audit comparing the **Legacy Heuristic Fronts (v0.6.x)** against the **Analytical Ground Truth (v0.7.5)**.

It demonstrates the jump in mathematical precision and identifies the geometric displacement of legacy truths.

In [ ]:
# 1. Setup Environment
import os
import sys
import pandas as pd
import numpy as np

if 'google.colab' in str(get_ipython()):
    if not os.path.exists('moeabench'):
        !git clone https://github.com/monacofj/moeabench.git
    else:
        # Always update to latest in branch
        %cd moeabench
        !git pull origin add-test
        %cd ..
    
    if os.path.exists('moeabench'):
        %cd moeabench
        !pip install -e .
    
    PROJECT_ROOT = os.getcwd()
else:
    # Local: Discover project root (one level up from misc/)
    # If running from root, use . as root
    if os.path.basename(os.getcwd()) == "misc":
        PROJECT_ROOT = os.path.dirname(os.getcwd())
    else:
        PROJECT_ROOT = os.getcwd()

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import MoeaBench as mb
from MoeaBench.core import SmartArray
print(f"Project Root defined at: {PROJECT_ROOT}")
print(f"MoeaBench version: {mb.system.version()}")

## Visual Auditor Helper
Logic to resolve absolute paths for artifacts and overlay data series.

In [ ]:
def audit_confrontation(mop_name, M=3):
    # 1. Setup Problem Instance (Official API)
    exp = mb.experiment()
    if mop_name.startswith("DPF"):
        mop = getattr(mb.mops, mop_name)(M=M, D=M-1)
    else:
        mop = getattr(mb.mops, mop_name)(M=M)
    
    # IMPORTANT: Assignment must occur before calling optimal()
    exp.mop = mop
    
    # 2. Setup New Truth (v0.7.5)
    try:
        f_gt_pop = exp.optimal(n_points=1000)
        F_gt = f_gt_pop.objectives # Use explicit Objectives SmartArray
    except (NotImplementedError, AttributeError):
        # Robust Fallback using absolute PROJECT_ROOT
        gt_file = os.path.join(PROJECT_ROOT, "tests/ground_truth", f"{mop_name}_{M}_optimal.csv")
        if os.path.exists(gt_file):
            F_gt = pd.read_csv(gt_file, header=None).values
        else:
            print(f"Critical: Analytical truth not found at {gt_file}.")
            return
            
    gt_series = SmartArray(F_gt, name="v0.7.5 Ground Truth", 
                           label="Referência Científica (v0.7.5)")
    
    # 3. Resolve and Load Legacy (v0.6.x)
    prefix = "lg__" if "DTLZ" in mop_name else "lg_"
    leg_filename = f"{prefix}{mop_name}_{M}_opt_front.csv"
    leg_path = os.path.join(PROJECT_ROOT, "tests/audit_data", f"legacy_{mop_name}", leg_filename)
    
    if os.path.exists(leg_path):
        F_leg = pd.read_csv(leg_path, header=None).values
        leg_series = SmartArray(F_leg, name="v0.6.x Legacy", 
                                label="Rastro Heurístico (v0.6.x)")
        
        # 4. View Confrontation: Overlay both series
        mb.view.topo_shape(gt_series, leg_series, 
                           title=f"{mop_name} (M={M}): Confronto de Verdades")
    else:
        print(f"Warning: Legacy data not found for {mop_name}. Plotting GT only.")
        mb.view.topo_shape(gt_series, title=f"{mop_name} (M={M}): Ground Truth Only")

## 1. DTLZ Family Audits

In [ ]:
audit_confrontation("DTLZ1")

In [ ]:
audit_confrontation("DTLZ2")

In [ ]:
audit_confrontation("DTLZ3")

In [ ]:
audit_confrontation("DTLZ4")

In [ ]:
audit_confrontation("DTLZ5")

In [ ]:
audit_confrontation("DTLZ6")

In [ ]:
audit_confrontation("DTLZ7")

In [ ]:
audit_confrontation("DTLZ8")

In [ ]:
audit_confrontation("DTLZ9")

## 2. DPF Family Audits

In [ ]:
audit_confrontation("DPF1")

In [ ]:
audit_confrontation("DPF2")

In [ ]:
audit_confrontation("DPF3")

In [ ]:
audit_confrontation("DPF4")

In [ ]:
audit_confrontation("DPF5")